# 阀门模型 (Valve Model)

`阀门 (Valve)` 类代表供水系统中的一个可控阀门。其主要功能是根据开启百分比来调节流量。流量是使用修正后的孔口方程计算的，该方程考虑了阀门两端的水头差。本Jupyter Notebook概述了 `Valve` 模型及其操作的仿真。

## 状态变量

阀门的状态由以下变量描述：

- `opening` (float): 阀门的当前开启度，以百分比表示 (0-100)。
- `outflow` (float): 计算出的通过阀门的流量 (m³/s)。

## 参数

阀门的物理属性由以下参数定义：

- `discharge_coefficient` (float): 最大流量系数（当阀门100%开启时）。
- `diameter` (float): 阀门所在管道的直径 (m)。

## 仿真示例

以下示例模拟了一个在不同控制信号下的阀门。阀门的开启度随时间调整，并根据恒定的水头差计算产生的出流量。仿真结果，包括开启百分比和出流量，将被绘制成图表。

**注意:** 要运行此Notebook，请确保您已安装 `matplotlib` (`pip install matplotlib`)，并且您是从项目的根目录运行Jupyter服务器的。

In [ ]:
import matplotlib.pyplot as plt
from swp.simulation_identification.physical_objects.valve import Valve
from swp.core.interfaces import State, Parameters

# 阀门的初始状态和参数
initial_state = State(opening=0.0, outflow=0.0)
parameters = Parameters(
    discharge_coefficient=0.85,
    diameter=0.3 # 300毫米
)

# 创建一个Valve实例
valve = Valve(name="control_valve", initial_state=initial_state, parameters=parameters)

# 仿真设置
dt = 1  # 时间步长（秒）
simulation_duration = 120
num_steps = int(simulation_duration / dt)

# 存储结果用于绘图
history = []

# 定义外部条件
upstream_head = 20.0
downstream_head = 15.0

# 运行仿真循环
for t in range(num_steps):
    # 随时间改变阀门开启度
    if t < 30:
        control_signal = 75 # 开启到75%
    elif t > 90:
        control_signal = 25 # 关闭到25%
    else:
        control_signal = 100 # 完全开启

    action = {
        'control_signal': control_signal,
        'upstream_head': upstream_head,
        'downstream_head': downstream_head
    }
    
    current_state = valve.step(action=action, dt=dt)
    history.append(current_state.copy())

# 打印最终状态
print("最终状态:", valve.get_state())

# 准备绘图数据
time_steps = [i * dt for i in range(num_steps)]
openings = [s['opening'] for s in history]
outflows = [s['outflow'] for s in history]

# 绘制结果
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

ax1.plot(time_steps, openings, label='阀门开启度 (Valve Opening)', drawstyle='steps-post')
ax1.set_ylabel('开启度 (%)')
ax1.set_title('阀门操作仿真')
ax1.grid(True)
ax1.legend()

ax2.plot(time_steps, outflows, label='出流量 (Outflow)', color='orange')
ax2.set_xlabel('时间 (秒)')
ax2.set_ylabel('出流量 (m³/s)')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()
